# Don't Upload

In [200]:
# necessary libraries:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from itertools import product
from datetime import date, timedelta
import calendar

# display settings
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 500

## Preparing the Data

In [201]:
df1 = pd.read_csv("data/campaign/01_campaign_level_performance-Jan-1-2018-Jun-30-2018.csv")
df2 = pd.read_csv("data/campaign/02_campaign_level_performance-Jul-1-2018-Dec-31-2018.csv")
df3 = pd.read_csv("data/campaign/03_campaign_level_performance-Jan-1-2019-Jun-30-2019.csv")
df4 = pd.read_csv("data/campaign/04_campaign_level_performance-Jul-1-2019-Dec-31-2019.csv")

C:\Users\demir\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [202]:
df = pd.concat([df1,df2,df3,df4])

In [203]:
df = df.sort_values(by=['Day'])

In [204]:
df['Amount Spent (USD)']=df['Amount Spent (USD)']*0.73

In [205]:
pd.to_datetime(df.Day).dt.strftime("%Y-%m").value_counts().sort_index()

2018-01    15662
2018-02    14328
2018-03    15692
2018-04    14701
2018-05    16910
2018-06    17973
2018-07    19457
2018-08    18871
2018-09    15410
2018-10    14906
2018-11    13910
2018-12    13813
2019-01    11189
2019-02    10253
2019-03    11736
2019-04    11984
2019-05    12784
2019-06     9639
2019-07     9842
2019-08    12092
2019-09    10277
2019-10     5872
2019-11     5495
2019-12     5493
Name: Day, dtype: int64

In [206]:
df.shape

(308289, 26)

In [207]:
#remove space and concat _column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('_-_', '_')

In [208]:
df=df[df['campaign_name'].str.contains('Facebook')]

In [209]:
df['campaign_name']=df.campaign_name.str.strip().str.replace(' ','')

In [210]:
df.campaign_name = df.campaign_name.str.replace("EMEA-","")
df.campaign_name = df.campaign_name.str.replace("LATAM-","")
df.campaign_name = df.campaign_name.str.replace("LatinAmerica-","")
df.campaign_name = df.campaign_name.str.replace("P1Areas-","")

In [211]:
df.campaign_name = df.campaign_name.str.replace("-old","")
df.campaign_name = df.campaign_name.str.replace("Facebook-","")
df.campaign_name = df.campaign_name.str.replace("P1Areas-","")
df.campaign_name = df.campaign_name.str.replace("-2018","")

In [212]:
df=df[df['campaign_name'].str.contains('LeadSubmission')]
df=df[df['campaign_name'].str.contains('PS|RRE')]

In [213]:
df=df[df['campaign_name']!='PS-LeadSubmissions']
df=df[df['campaign_name']!='RRE-LeadSubmissions']

In [214]:
df['campaign_name'] = df['campaign_name'].str.replace("PS","OR")
df['campaign_name'] = df['campaign_name'].str.replace("RRE","SSF")

In [215]:
df=df.rename(columns={'amount_spent_(usd)':'amount_spent_usd'})

In [216]:
df['location'] = df.campaign_name.str.split('-').str[0]
df['industry'] = df.campaign_name.str.split('-').str[1]

In [217]:
df.reset_index(drop=True,inplace=True)

In [218]:
df.drop(columns=['campaign_id',
                 'campaign_name',
                 'delivery_status',
                 'delivery_level',
                 'uk_thank_you',
                 'lead_submission',
                 'lead_submission_conversion_value',
                 'lead_page_contains_thank-you_conversion_value',
                 'reporting_starts',
                 'reporting_ends',
                 'unique_landing_page_views',
                 'page_engagement',
                 'cpc_(cost_per_link_click)',
                 'adds_to_cart',
                 'campaign_budget',
                 'campaign_budget_type',
                 'campaign_budget',],inplace=True)

In [219]:
df=df.fillna(0)

In [220]:
df.columns = ['day',
 'time_of_day',
 'amount_spent_usd',
 'impressions',
 'landing_page_views',
 'conversion',
 'page_likes',
 'post_comments',
 'post_engagement',
 'post_reactions',
 'location',
 'industry']

In [221]:
df = df[['day',
 'time_of_day',
 'amount_spent_usd',
 'impressions',
 'landing_page_views',
 'page_likes',
 'post_comments',
 'post_engagement',
 'post_reactions',
 'location',
 'industry',
 'conversion']]

In [222]:
df.drop_duplicates(keep=False,inplace=True)

In [223]:
df=df[df['location'].isin(['US', 'FR', 'UK', 'MX', 'ES', 'DE', 'ZA', 'IT'])].reset_index(drop=True)

In [224]:
df['time_of_day']=df['time_of_day'].str.split(':').str[0].astype(int)

In [226]:
df=df[pd.to_datetime(df['day'])<'2019-10']

In [227]:
df.head()

,day,time_of_day,amount_spent_usd,impressions,landing_page_views,page_likes,post_comments,post_engagement,post_reactions,location,industry,conversion
0,2018-01-01,5,0.7008,203,1.0,0.0,0.0,16.0,1.0,ZA,OR,0.0
1,2018-01-01,6,0.6570,299,2.0,0.0,0.0,31.0,0.0,ZA,OR,2.0
2,2018-01-01,7,1.1461,551,3.0,0.0,0.0,73.0,0.0,ZA,OR,0.0
3,2018-01-01,8,1.3870,554,2.0,0.0,0.0,68.0,1.0,ZA,OR,0.0
4,2018-01-01,9,1.2629,537,3.0,1.0,0.0,62.0,0.0,ZA,OR,0.0


In [228]:
df.reset_index(drop=True,inplace=True)

In [229]:
df.to_csv('data/campaign/fb_campaign.csv',index=False)